<a target="_blank" href="https://colab.research.google.com/github/bettercodepaul/nixtla_intro_workshop/blob/main/Introduction_to_TimeGPT.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Forecasting with Nixtla's TimeGPT

This notebook walks you through the very basics of forecasting time series with Nixtla's foundation model TimeGPT.

## Install and import necessary libraries

We use [Polars](https://docs.pola.rs/) for data wrangling, [Plotly](https://plotly.com/python/plotly-express/) for visualizations and Nixtla's [TimeGPT](https://www.nixtla.io/docs/introduction/introduction) for basic time series forecasting.

In [ ]:
pip -q install nixtla polars plotly

In [ ]:
import polars as pl
import plotly.express as px
from datetime import date

In [ ]:
from google.colab import userdata

## Create a TimeGPT account and generate your API key

- Visit [dashboard.nixtla.io](https://dashboard.nixtla.io) to activate your free trial and create an account.
- Sign in using Google, GitHub, or your email.
- Navigate to API Keys in the menu and select Create New API Key.
- Your new API key will appear on the screen. Copy this key using the button on the right.
- Add it to your Google Colab secrets on the left

In [ ]:
from nixtla import NixtlaClient

nixtla_client = NixtlaClient(
    api_key=userdata.get('NIXTLA_API_KEY')
)
nixtla_client.validate_api_key()

## Load the data

The data for this walk through is simple monthly sales data from various countries.

In [ ]:
df = pl.read_parquet("https://github.com/bettercodepaul/nixtla_intro_workshop/raw/refs/heads/main/retail_sales.parquet")
df.sample(5)

In [ ]:
nixtla_client.plot(df, time_col='date', target_col='sales', id_col="country")

In [ ]:
# Make a forecast
timegpt_fcst_df = nixtla_client.forecast(
    df=df,
    h=12,
    freq="1mo",
    time_col="date",
    target_col="sales",
    id_col="country"
)
timegpt_fcst_df.head()

In [ ]:
# Plot the forecast
nixtla_client.plot(df, timegpt_fcst_df, time_col="date", target_col="sales", id_col="country")

# Hands-on

- Try different cut-offs and check if it gets better to catch the seasonality.
- Compare with the results from the first notebook
- Try what you always wanted to try using time series!

In [ ]:
# you can code here!